# Introduction

First submission :

<a href='#1'>1. Loading Data</a>

<a href='#2'>2. Combining Datasets</a>

<a href='#3'>3. Memory Limitation</a>

<a href='#4'>4. Missing Values</a>

<a href='#6'>5. Model Training</a>

<a href='#7'>6. Model Predictions and Kaggle Submission</a>

# <a id='1'>1. Loading Data</a>

In [1]:
#import libraries
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
#load data
train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
train['timestamp'] = pd.to_datetime(train['timestamp']) #the train dataset contains a 'timestamp' column we convert to a datetime object for ease of use
test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')
test['timestamp'] = pd.to_datetime(test['timestamp'])
weather_train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp']) 
weather_test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')
weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp']) 
build_meta = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')

In [18]:
train.shape

(20216100, 4)

In [19]:
test.shape

(41697600, 4)

For a simple first model, we are going to merge the training sets to gather all covariables and make predictions on it with a linear regressor.

# <a id='2'>2. Combining Datasets</a>
We will merge everything into train and test dataframes.

We need to convert the timestamp to the right type:

In [3]:
weather_test.timestamp = pd.to_datetime(weather_test.timestamp) 
weather_train.timestamp = pd.to_datetime(weather_train.timestamp) 

In [4]:
#merge the building meta data and weather data into the train data
train_m = train.merge(build_meta, how='left', on = ['building_id'], validate='many_to_one') #merge the building meta data into the train data
test_m = test.merge(build_meta, how='left', on = ['building_id'], validate='many_to_one') #merge the building meta data into the train data
train_m = train_m.merge(weather_train, how='left', on = ['site_id', 'timestamp'], validate='many_to_one')#add weather data to each time entry for each site ID
test_m = test_m.merge(weather_test, how='left', on = ['site_id', 'timestamp'], validate='many_to_one')#add weather data to each time entry for each site ID
del build_meta, weather_train, weather_test
import gc
gc.collect() #mandatory since we don't have unlimited space (16go and the databases are quite large)
train_m.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


# <a id='3'>3. Memory Limitation</a>

In [5]:
def reduce_mem_usage(df, verbose=True): #we need to drastically reduce memory usage to work with those data basis
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_m = reduce_mem_usage(train_m) #this code changes the variable formats so it takes less space
test_m = reduce_mem_usage(test_m) #source for this code : https://www.kaggle.com/alexandersylvester/ashrae-energy-predictions-with-lightgbm

Mem. usage decreased to 1041.10 Mb (60.3% reduction)
Mem. usage decreased to 2147.36 Mb (60.3% reduction)


# <a id='4'>4. Missing Values</a>

In [27]:
train_m.isna().describe()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet
count,20216100,20216100,20216100,20216100,20216100,20216100,20216100
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,20216100,20216100,20216100,20216100,20216100,20216100,20216100


In [9]:
test_m.isna().describe()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600,41697600
unique,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False
freq,41697600,41697600,41697600,41697600,41697600,41697600,41697600,24598080,34444320,41475699,22155420,41436801,33896037,39180774,38718937,41395511


The easy way out in this scenario is to drop the columns containing any NA. Which we're going to do for the time being. We have no missing value for the target variable so we will not be dropping any training row

In [10]:
train_m = train_m.dropna(axis='columns')

In [11]:
gc.collect(generation=0)
gc.collect(generation=1)
gc.collect() #supposedly cleans the memory but not that efficient // we need a better way

0

In [13]:
test_m = test_m.dropna(axis='columns')

In [22]:
train_m.describe()

,building_id,meter,meter_reading,site_id,square_feet
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07
mean,7.992780e+02,6.624412e-01,1.988706e+03,7.992232e+00,1.077830e+05
std,4.269133e+02,9.309921e-01,1.532159e+05,5.099060e+00,1.171424e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02
25%,3.930000e+02,0.000000e+00,1.830000e+01,3.000000e+00,3.252700e+04
50%,8.950000e+02,0.000000e+00,7.877500e+01,9.000000e+00,7.270900e+04
75%,1.179000e+03,1.000000e+00,2.679840e+02,1.300000e+01,1.391130e+05
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05


In [23]:
test_m.describe()

,row_id,building_id,meter,site_id,square_feet
count,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07
mean,2.084880e+07,8.075824e+02,6.642857e-01,8.086134e+00,1.069469e+05
std,1.203706e+07,4.297680e+02,9.278067e-01,5.134712e+00,1.160888e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02
25%,1.042440e+07,4.047500e+02,0.000000e+00,3.000000e+00,3.224350e+04
50%,2.084880e+07,9.000000e+02,0.000000e+00,9.000000e+00,7.226250e+04
75%,3.127320e+07,1.194250e+03,1.000000e+00,1.300000e+01,1.383875e+05
max,4.169760e+07,1.448000e+03,3.000000e+00,1.500000e+01,8.750000e+05


In [31]:
test_m = test_m.drop('row_id',axis=1)
test_m.describe()

KeyError: "['row_id'] not found in axis"

# <a id='5'>5. Model Training</a>

In [33]:
%%time
from sklearn.model_selection import GridSearchCV #, train_test_split,TimeSeriesSplit
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor

y_train = train_m['meter_reading']

my_tree = DecisionTreeRegressor()

MSE = make_scorer(mean_squared_error)

param_grid = {'splitter' : ['best', 'random'],'criterion' : ['mse', 'friedman_mse', 'mae', 'poisson']}
model = GridSearchCV(my_tree, param_grid, scoring=MSE)
model.fit(train_m.drop('meter_reading',axis=1) , y_train)
predicted = model.predict(test_m)
print('Best Parameters found for MSE : ',model.best_params_)
print('Train Score : ',model.score(train_m.drop('meter_reading') , y_train))



/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 1246, in fit
    X_idx_sorted=X_idx_sorted)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 158, in fit
    check_y_params))
  File "/opt/conda/lib/python3.7/site-packages/sklearn/base.py", line 429, in _validate_data
    X = check_array(X, **check_X_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py", line 533, in check_a

TypeError: invalid type promotion

In [30]:
y_train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: meter_reading, dtype: float32

Now that we have tuned the model parameters and have an idea of model performance. We will fit on the entire training dataset so we have as much information as possible for the final test set prediction.

# <a id='6'>6. Model Predictions and Kaggle Submission</a>

In [ ]:
submission.to_csv('../submission.csv', index=False)
print('Submission CSV is ready!')